$\#\#\#\#\#\#\#$ Llibreries $\#\#\#\#\#\#\#$

In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

import warnings

warnings.filterwarnings('ignore') 

$\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#$

%

# Data

We load the cleaned data set from exercise 5:

In [3]:
df = pd.read_csv('ufc-strikes-by-round-data_CLEAN.csv')

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.head()

,roundinfo_id,fight_id,fight_round,fighter_id,kd,sig_str_made,sig_str_attempted,sig_str_percent,total_str_made,total_str_attempted,...,head_made,body_attempted,body_made,leg_attempted,leg_made,distance_attempted,distance_made,clinch_attempted,clinch_made,ground_attempted
0,9532,1,5130,1,14,28,50,14,28,0,...,1,1,1,1,24,11,4,3,0,0
1,9532,1,5131,0,10,19,52,10,19,0,...,1,1,4,3,15,8,4,2,0,0
2,9533,1,5132,0,10,16,62,28,36,0,...,2,2,0,0,13,8,1,1,2,1
3,9533,1,5133,0,44,72,61,64,111,3,...,1,1,2,1,7,2,1,1,64,41
4,9533,2,5132,0,7,11,63,12,16,0,...,2,2,0,0,11,7,0,0,0,0


To complete this homework we will use the test from theoretical information on the Academia website:
https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/.

When we are using null hypothesis and p-values it is easy to overestimate the sample. With fMRI data, for example, if we want to infer a network and we use these estimates, in some cases we will lose or win connections on the network that might be relavant for the fact we are estimating values over a threshold (usually 0.05) that might not contemplate that a link on this case might be caused by and indirect connection between two other nodes or, that we are accepting flucutations in the dataset as true links. This is why in research before picking a alpha value equal to 0.05 they study a range of alphas and determine the more adequate for the type of system they are studying. In the fMRI networks, the p value picked is usually around 0.15, this value discards most of the fluctuations, [1], and it takes into account the nature of the link (direct/indirect connection).

[1] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1904171/

# Nivell 1

## Exercici 1: Agafa un conjunt de dades de tema esportiu que t'agradi i selecciona un atribut del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%. 

In [26]:
# pvalue with tTest:
# I will choose single strike attempted. Hypothesis: The single strikes attempted are equal to the mean of the
# population of the strikes attempted.
popmean = df.sig_str_attempted.mean()
tset, pval = stats.ttest_1samp(df.sig_str_attempted, popmean)

if pval < 0.05:
    print("Null Hypothesis rejected.")
else:
    print("Null Hypothesis accepted.")

Null Hypothesis accepted.


Of course, since we have choosen the t-test to compare it with the mean of the feature, the hypothesis is accepted.

# Nivell 2

## Exercici 2: Continua amb el conjunt de dades de tema esportiu que t'agradi i selecciona dos altres atributs del conjunt de dades. Calcula els p-valors i digues si rebutgen la hipòtesi nul·la agafant un alfa de 5%.


In [7]:
# Chi square test:

# Are dependents the strikes made with the strikes on legs?

table = pd.crosstab(df.total_str_made, df.leg_made)
stat, pval, dof, expected = stats.chi2_contingency(table)

if pval <= 0.05:
    print("Dependents. Accepts Null Hypothesis.")
else:
    print("Independents. Rejects Null Hypothesis.")

Dependents. Accepts Null Hypothesis.


They are dependent because part of the total count of strikes made, some must have landed on the rival fighter legs.

In [25]:
# We can try to the same with Pearson's correlation:
x, pval = stats.pearsonr(df.total_str_made, df.leg_made)
print('correlation between them:{:.2f}'.format(x))

# now we need to apply the p-value over each one of the values in the correlation matrix.
if pval <= 0.05:
    print("Dependents. Accepts Null Hypothesis.")
else:
    print("Independents. Reject Null Hypothesis.")

correlation between them:0.74
Dependents. Accepts Null Hypothesis.


# Nivell 3

## Exercici 3: Continua amb el conjunt de dades de tema esportiu que t'agradi i selecciona tres atributs del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%.

In [15]:
# ANOVA test:
#Null hypothesis: The strikes made are the ones recibed in the head and in the body? (are dependents)

stat, pval = stats.f_oneway(df.total_str_made, df.head_made, df.body_made)

if pval > 0.05:
    print("Dependents. We accept null hypothesis.")
else:
    print("Independents. We reject null hypothesis.")


Independents. We reject null hypothesis.


This result has easy interpretation, while total strikes made is dependent of head strikes and body strikes, head strikes and body strikes are independent, since a fighter that hits on the head not increases the value of hits in the body. The same will have applied if we were using the variable legs strikes instead of body or head, while mantaining the other in the analysis.